In [ ]:

from transformers import pipeline
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import torch
import os
!pip install nltk tqdm
import nltk, re, pandas as pd
from tqdm import tqdm
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/CS4650/project'

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
login(token="") #removed out for security

In [ ]:
device = 0 if torch.cuda.is_available() else -1

# WNC dataset processing

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[hf-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter


file_path = os.path.join(folder_path, "data/WNC/WNC/biased.word.train")


# # Load the latest version
# hf_dataset = kagglehub.load_dataset(
#   KaggleDatasetAdapter.HUGGING_FACE,
#   "chandiragunatilleke/wiki-neutrality-corpus",
#   file_path,
#   # Provide any additional arguments like
#   # sql_query, hf_kwargs, or pandas_kwargs. See
#   # the documenation for more information:
#   # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterhugging_face
# )
# print("Hugging Face Dataset:", hf_dataset)

df_mcn = pd.read_csv(
    file_path,
    sep="\t",
    header=None,
    names=["id","src_tok","tgt_tok","biased","neutral","src_POS","tgt_PARSE"],
    usecols=["biased","neutral"]
)

print(df_mcn.shape)
display(df_mcn.head(-10))



(53803, 2)


,biased,neutral
0,"chloroform ""the molecular lifesaver"" an articl...","chloroform ""the molecular lifesaver"" an articl..."
1,the free software gnu classpath project is onl...,the free software gnu classpath project is par...
2,"other campaigners, especially the controversia...","other campaigners, especially the british acti..."
3,vocalist rob halford's performance is consider...,vocalist rob halford's performance is consider...
4,the proud general is a chinese animated featur...,the proud general is a chinese animated featur...
...,...,...
53788,these books are considered to be part of old t...,these books are considered to be part of old t...
53789,"in terms of public transport, cumbernauld has ...","in terms of public transport, cumbernauld has ..."
53790,"like the rest of nature, man's mind is subject...","like the rest of nature, man's mind is subject..."
53791,the band are very popular for their 3d animate...,the band are popular for their 3d animated vid...


## classifier model

In [ ]:
classifier = pipeline("text-classification", model="GroNLP/mdebertav3-subjectivity-english", device=device)

Device set to use cpu


In [ ]:
sentence = "vocalist rob halford's performance is considered one of his finest ever, and the guitar work is noted as well, especially the supremely epic dual guitar solo."
result_food = classifier(sentence)
print(result_food)
# label_1: subjective; label_0: objective

[{'label': 'LABEL_1', 'score': 0.6280233263969421}]


In [ ]:
from transformers import AutoConfig

cfg = AutoConfig.from_pretrained("GroNLP/mdebertav3-subjectivity-english")
print(cfg.id2label)
print(cfg.label2id)

{0: 'LABEL_0', 1: 'LABEL_1'}
{'LABEL_0': 0, 'LABEL_1': 1}


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

name = "GroNLP/mdebertav3-subjectivity-english"
tok = AutoTokenizer.from_pretrained(name)
model = AutoModelForSequenceClassification.from_pretrained(name)

for s in [
    "The government announced a new policy today.",   # objective
    "This is a horrible and unfair decision!"          # subjective
]:
    t = tok(s, return_tensors="pt")
    with torch.no_grad():
        probs = model(**t).logits.softmax(dim=-1)
    print(s)
    print("probabilities:", probs.tolist())


The government announced a new policy today.
probabilities: [[0.9522690176963806, 0.04773103818297386]]
This is a horrible and unfair decision!
probabilities: [[0.16840575635433197, 0.8315942287445068]]


In [ ]:
lexicon_path=os.path.join(folder_path, "data/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff")
def load_mpqa_lexicon(path):
    subj_dict = {}
    with open(path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            entry = {}
            for p in parts:
                if '=' in p:
                    key, value = p.split('=', 1)
                    entry[key] = value
            word = entry.get("word1", "").lower()
            subj_type = entry.get("type", "")
            polarity = entry.get("priorpolarity", "")
            if word:
                subj_dict[word] = {"type": subj_type, "polarity": polarity}
    return subj_dict
mpqa_dict = load_mpqa_lexicon(lexicon_path)
print("Sample:", list(mpqa_dict.items())[:10])
types = sorted({v["type"] for v in mpqa_dict.values()})
polarities = sorted({v["polarity"] for v in mpqa_dict.values()})
print("Types: ", types)
print("polarities: ", polarities)

Sample: [('abandoned', {'type': 'weaksubj', 'polarity': 'negative'}), ('abandonment', {'type': 'weaksubj', 'polarity': 'negative'}), ('abandon', {'type': 'weaksubj', 'polarity': 'negative'}), ('abase', {'type': 'strongsubj', 'polarity': 'negative'}), ('abasement', {'type': 'strongsubj', 'polarity': 'negative'}), ('abash', {'type': 'strongsubj', 'polarity': 'negative'}), ('abate', {'type': 'weaksubj', 'polarity': 'negative'}), ('abdicate', {'type': 'weaksubj', 'polarity': 'negative'}), ('aberration', {'type': 'strongsubj', 'polarity': 'negative'}), ('abhor', {'type': 'strongsubj', 'polarity': 'negative'})]
Types:  ['strongsubj', 'weaksubj']
polarities:  ['both', 'negative', 'neutral', 'positive', 'weakneg']


In [ ]:

def tag_subjectivity(sentence, lexicon):
    tokens = word_tokenize(sentence)
    flags = []
    for tok in tokens:
        w = tok.lower()
        if w in lexicon:
            subj_type = lexicon[w]["type"]
            flags.append(1 if subj_type in ["strongsubj"] else 0) #"weaksubj"
        else:
            flags.append(0)
    return tokens, flags
print(tag_subjectivity("chloroform 'the molecular lifesaver' an article at oxford university providing interesting facts about chloroform.", mpqa_dict))

(['chloroform', "'the", 'molecular', 'lifesaver', "'", 'an', 'article', 'at', 'oxford', 'university', 'providing', 'interesting', 'facts', 'about', 'chloroform', '.'], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0])


In [ ]:
tqdm.pandas()

df_mcn["tokens_flags"] = df_mcn["biased"].progress_apply(
    lambda x: tag_subjectivity(x, mpqa_dict)
)

# Split the tuple column into two separate columns
df_mcn["tokens"] = df_mcn["tokens_flags"].apply(lambda x: x[0])
df_mcn["flags"] = df_mcn["tokens_flags"].apply(lambda x: x[1])

# Drop the temporary column
df_mcn = df_mcn.drop(columns=["tokens_flags"])

100%|██████████| 53803/53803 [00:11<00:00, 4562.57it/s]


In [ ]:
display(df_mcn[["biased", "neutral", "tokens", "flags"]].head(5))

,biased,neutral,tokens,flags
0,"chloroform ""the molecular lifesaver"" an articl...","chloroform ""the molecular lifesaver"" an articl...","[chloroform, ``, the, molecular, lifesaver, ''...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
1,the free software gnu classpath project is onl...,the free software gnu classpath project is par...,"[the, free, software, gnu, classpath, project,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"other campaigners, especially the controversia...","other campaigners, especially the british acti...","[other, campaigners, ,, especially, the, contr...","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,vocalist rob halford's performance is consider...,vocalist rob halford's performance is consider...,"[vocalist, rob, halford, 's, performance, is, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,the proud general is a chinese animated featur...,the proud general is a chinese animated featur...,"[the, proud, general, is, a, chinese, animated...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
import os

output_path = os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/biased_word_train.csv")
output_dir = os.path.dirname(output_path)

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_mcn.to_csv(output_path, index=False)
print("Saved:", output_path)

Saved: /content/drive/MyDrive/CS4650/project/data/WNC/processed/mpqa_lexicon/biased_word_train.csv


# Load Processed WNC (mpqa_lexicon flagged)

path=data/WNC/processed/mpqa_lexicon/biased_word_train.csv

In [ ]:
mcn_processed_path=os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/biased_word_train.csv")
df=pd.read_csv(mcn_processed_path)
display(df)

,biased,neutral,tokens,flags
0,"chloroform ""the molecular lifesaver"" an articl...","chloroform ""the molecular lifesaver"" an articl...","['chloroform', '``', 'the', 'molecular', 'life...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
1,the free software gnu classpath project is onl...,the free software gnu classpath project is par...,"['the', 'free', 'software', 'gnu', 'classpath'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"other campaigners, especially the controversia...","other campaigners, especially the british acti...","['other', 'campaigners', ',', 'especially', 't...","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,vocalist rob halford's performance is consider...,vocalist rob halford's performance is consider...,"['vocalist', 'rob', 'halford', ""'s"", 'performa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,the proud general is a chinese animated featur...,the proud general is a chinese animated featur...,"['the', 'proud', 'general', 'is', 'a', 'chines...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
53798,the national lawyers guild is a progressive /l...,the national lawyers guild is a progressive ba...,"['the', 'national', 'lawyers', 'guild', 'is', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
53799,a plan to redevelop the old tiger stadium site...,a plan to redevelop the old tiger stadium site...,"['a', 'plan', 'to', 'redevelop', 'the', 'old',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
53800,"instrumentally, lifeson is regarded as a virtu...","instrumentally, lifeson is regarded as a guita...","['instrumentally', ',', 'lifeson', 'is', 'rega...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
53801,flynt joined the us army in 1958 at only fifte...,flynt joined the us army in 1958 at only fifte...,"['flynt', 'joined', 'the', 'us', 'army', 'in',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
def format_input(tokens, flags):
    words = tokens.strip("[]").split(", ")
    flags = flags.strip("[]").split(", ")
    words = [w.replace("'", "") for w in words]
    combined = " ".join([f"{w}[{f}]" for w, f in zip(words, flags)])
    return "neutralize: " + combined

df["input_text"] = [format_input(t, f) for t, f in zip(df["tokens"], df["flags"])]
df["target_text"] = df["neutral"]
display(df)

,biased,neutral,tokens,flags,input_text,target_text
0,"chloroform ""the molecular lifesaver"" an articl...","chloroform ""the molecular lifesaver"" an articl...","['chloroform', '``', 'the', 'molecular', 'life...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...",neutralize: chloroform[0] ``[0] the[0] molecul...,"chloroform ""the molecular lifesaver"" an articl..."
1,the free software gnu classpath project is onl...,the free software gnu classpath project is par...,"['the', 'free', 'software', 'gnu', 'classpath'...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: the[0] free[0] software[0] gnu[0] ...,the free software gnu classpath project is par...
2,"other campaigners, especially the controversia...","other campaigners, especially the british acti...","['other', 'campaigners', ',', 'especially', 't...","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","neutralize: other[0] campaigners[0] ,[0] espec...","other campaigners, especially the british acti..."
3,vocalist rob halford's performance is consider...,vocalist rob halford's performance is consider...,"['vocalist', 'rob', 'halford', ""'s"", 'performa...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","neutralize: vocalist[0] rob[0] halford[0] ""s""[...",vocalist rob halford's performance is consider...
4,the proud general is a chinese animated featur...,the proud general is a chinese animated featur...,"['the', 'proud', 'general', 'is', 'a', 'chines...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: the[0] proud[1] general[0] is[0] a...,the proud general is a chinese animated featur...
...,...,...,...,...,...,...
53798,the national lawyers guild is a progressive /l...,the national lawyers guild is a progressive ba...,"['the', 'national', 'lawyers', 'guild', 'is', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: the[0] national[0] lawyers[0] guil...,the national lawyers guild is a progressive ba...
53799,a plan to redevelop the old tiger stadium site...,a plan to redevelop the old tiger stadium site...,"['a', 'plan', 'to', 'redevelop', 'the', 'old',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: a[0] plan[0] to[0] redevelop[0] th...,a plan to redevelop the old tiger stadium site...
53800,"instrumentally, lifeson is regarded as a virtu...","instrumentally, lifeson is regarded as a guita...","['instrumentally', ',', 'lifeson', 'is', 'rega...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","neutralize: instrumentally[0] ,[0] lifeson[0] ...","instrumentally, lifeson is regarded as a guita..."
53801,flynt joined the us army in 1958 at only fifte...,flynt joined the us army in 1958 at only fifte...,"['flynt', 'joined', 'the', 'us', 'army', 'in',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",neutralize: flynt[0] joined[0] the[0] us[0] ar...,flynt joined the us army in 1958 at only fifte...


In [ ]:
output_path = os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/input_target_pair_biased_word_train.csv")
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df[["input_text", "target_text"]].to_csv(output_path, index=False)
print("Saved:", output_path)

Saved: /content/drive/MyDrive/CS4650/project/data/WNC/processed/mpqa_lexicon/input_target_pair_biased_word_train.csv


# fine tune model (cross entropy loss & classifier loss)

In [ ]:
import torch
from torch import nn
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
dataset = load_dataset("csv", data_files=os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/input_target_pair_biased_word_train.csv"))

max_length = 128
def preprocess(batch):
    model_inputs = tokenizer(batch["input_text"], max_length=max_length, truncation=True, padding=True)
    labels = tokenizer(batch["target_text"], max_length=max_length, truncation=True, padding=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)
print(tokenized_dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/53803 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 53803
    })
})


In [ ]:
# @title loss function
classifier_name = "GroNLP/mdebertav3-subjectivity-english"
clf_tokenizer = AutoTokenizer.from_pretrained(classifier_name)
clf_model = AutoModelForSequenceClassification.from_pretrained(classifier_name)
clf_model.eval()
for p in clf_model.parameters():
  p.requires_grad = False

class DualLossTrainer(Seq2SeqTrainer):
  def __init__(self, classifier, clf_tokenizer, lambda_obj=0.2, **kwargs):
      super().__init__(**kwargs)
      self.classifier = classifier
      self.clf_tokenizer = clf_tokenizer
      self.lambda_obj = lambda_obj
      self.ce_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
      # Move classifier and tokenizer to the same device as the model
      self.classifier.to(self.model.device)
      # The tokenizer itself doesn't need a device, but its outputs should be on the model's device.

  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
      # labels = inputs.pop("labels")
      # print(inputs["labels"][0][:20])

      outputs = model(**inputs)
      logits = outputs.logits
      labels=inputs["labels"]
      # print(logits)

      # if torch.isnan(logits).any() or torch.isinf(logits).any():
      #   print("🚨 Logits contain NaN or Inf before CE Loss! Stopping.")
      #   return total_loss # Or raise an error

      # ---------- Cross-entropy loss ----------
      ce_loss = self.ce_loss_fn(
        logits.view(-1, logits.size(-1)),
        labels.view(-1)
      )


      # ---------- Objectivity loss ----------
      # Generate sentences with greedy decoding (no gradient)
      with torch.no_grad():
        generated = torch.argmax(logits, dim=-1)
        # decoded = self.tokenizer.batch_decode(
        #     generated, skip_special_tokens=True
        # )
        decoded = self.processing_class.batch_decode(
          generated, skip_special_tokens=True
        )


        # Run classifier on generated text
        scores = []
        for text in decoded:
            tokens = self.clf_tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=128,
            ).to(model.device)
            pred = self.classifier(**tokens).logits.softmax(dim=-1)
            # index 0=objective, index1=subjective
            score = pred[0][0]   # probability of "objective"
            scores.append(score)
      scores = torch.stack(scores)
      obj_loss = (1 - scores).mean()

      total_loss = (1 - self.lambda_obj) * ce_loss + self.lambda_obj * obj_loss
      print(f"CE: {ce_loss.item():.4f}, OBJ: {obj_loss.item():.4f}, TOTAL: {total_loss.item():.4f}")
      return (total_loss, outputs) if return_outputs else total_loss

In [ ]:

# @title model & args
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

training_args = Seq2SeqTrainingArguments(
    output_dir=os.path.join(folder_path, "model_checkpoints/wnc-mpqa-flan_t5-2loss-11_09"),
    save_strategy="steps",
    save_steps=1000,
    # evaluation_strategy="steps",
    # eval_steps=1000,
    learning_rate=1e-5, #1e-5 to 5e-5
    num_train_epochs=1, #3
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=False,
    bf16=False,
    logging_dir=os.path.join(folder_path, "logs/wnc-mpqa-flan_t5-2loss-11_09"),
)

trainer = DualLossTrainer(
    model=model,
    processing_class=tokenizer,
    classifier=clf_model,
    clf_tokenizer=clf_tokenizer,
    lambda_obj=0.2,  # 20 % weight on objectivity loss
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"].select(range(500)),
)

In [ ]:
# @title training
trainer.train()

CE: 34.1808, OBJ: 0.6277, TOTAL: 27.4702


Step,Training Loss
500,4.124200
1000,0.393300
1500,0.345300
2000,0.328700
2500,0.344400
3000,0.339900
3500,0.326700
4000,0.324400
4500,0.320600
5000,0.332800


流式输出内容被截断，只能显示最后 5000 行内容。
CE: 0.4332, OBJ: 0.3204, TOTAL: 0.4107
CE: 0.4072, OBJ: 0.3866, TOTAL: 0.4031
CE: 0.5279, OBJ: 0.2391, TOTAL: 0.4702
CE: 0.3595, OBJ: 0.2950, TOTAL: 0.3466
CE: 0.2764, OBJ: 0.3480, TOTAL: 0.2908
CE: 0.1843, OBJ: 0.2926, TOTAL: 0.2060
CE: 0.4434, OBJ: 0.2181, TOTAL: 0.3984
CE: 0.4825, OBJ: 0.3791, TOTAL: 0.4618
CE: 0.6825, OBJ: 0.4539, TOTAL: 0.6367
CE: 0.3524, OBJ: 0.2513, TOTAL: 0.3322
CE: 0.4400, OBJ: 0.4324, TOTAL: 0.4384
CE: 0.2929, OBJ: 0.3152, TOTAL: 0.2973
CE: 0.5727, OBJ: 0.3388, TOTAL: 0.5259
CE: 0.3536, OBJ: 0.3199, TOTAL: 0.3469
CE: 0.2697, OBJ: 0.3119, TOTAL: 0.2781
CE: 0.5362, OBJ: 0.3438, TOTAL: 0.4977
CE: 0.4338, OBJ: 0.3346, TOTAL: 0.4139
CE: 0.2177, OBJ: 0.3926, TOTAL: 0.2527
CE: 0.0895, OBJ: 0.3264, TOTAL: 0.1369
CE: 0.2737, OBJ: 0.3391, TOTAL: 0.2868
CE: 0.1591, OBJ: 0.4363, TOTAL: 0.2146
CE: 0.1503, OBJ: 0.3521, TOTAL: 0.1907
CE: 0.3289, OBJ: 0.3792, TOTAL: 0.3390
CE: 0.3858, OBJ: 0.3257, TOTAL: 0.3738
CE: 0.3035, OBJ: 0.3174, TOTAL: 0.306

TrainOutput(global_step=6726, training_loss=0.6164255496684973, metrics={'train_runtime': 3141.2755, 'train_samples_per_second': 17.128, 'train_steps_per_second': 2.141, 'total_flos': 9210499958439936.0, 'train_loss': 0.6164255496684973, 'epoch': 1.0})

# test

In [ ]:
test_input = "The[0] horrible[1] dictator[0] destroyed[0] the[0] peaceful[1] town[0]."
enc = tokenizer(test_input, return_tensors="pt").to(model.device)
out = model.generate(**enc, max_new_tokens=64)
print(tokenizer.decode(out[0], skip_special_tokens=True))

the drunky emperor destroyed the peaceful town


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_path = os.path.join(folder_path,
    "model_checkpoints/wnc-mpqa-flan_t5-2loss-11_09/checkpoint-6000"
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
trained_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
trained_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
text = "neutralize: The[0] horrible[1] dictator[0] destroyed[0] the[0] peaceful[1] town[0]."

inputs = tokenizer(text, return_tensors="pt").to(trained_model.device)
with torch.no_grad():
    outputs = trained_model.generate(
        **inputs,
        max_new_tokens=64,
        num_beams=4,      # optional: beam search
        do_sample=False,  # deterministic
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The dictator destroyed the peaceful town.


## WNC test data

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[hf-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter


file_path = os.path.join(folder_path, "data/WNC/WNC/biased.word.test")
df_mcn = pd.read_csv(
    file_path,
    sep="\t",
    header=None,
    names=["id","src_tok","tgt_tok","biased","neutral","src_POS","tgt_PARSE"],
    usecols=["biased","neutral"]
)

lexicon_path=os.path.join(folder_path, "data/subjectivity_clues_hltemnlp05/subjclueslen1-HLTEMNLP05.tff")
def load_mpqa_lexicon(path):
    subj_dict = {}
    with open(path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            entry = {}
            for p in parts:
                if '=' in p: # Check if '=' exists in the part
                    key, value = p.split('=', 1) # Split only once
                    entry[key] = value
            word = entry.get("word1", "").lower()
            subj_type = entry.get("type", "")
            polarity = entry.get("priorpolarity", "")
            if word:
                subj_dict[word] = {"type": subj_type, "polarity": polarity}
    return subj_dict
mpqa_dict = load_mpqa_lexicon(lexicon_path)

def tag_subjectivity(sentence, lexicon):
    tokens = word_tokenize(sentence)
    flags = []
    for tok in tokens:
        w = tok.lower()
        if w in lexicon:
            subj_type = lexicon[w]["type"]
            flags.append(1 if subj_type in ["strongsubj"] else 0) #"weaksubj"
        else:
            flags.append(0)
    return tokens, flags
tqdm.pandas()

df_mcn["tokens_flags"] = df_mcn["biased"].progress_apply(
    lambda x: tag_subjectivity(x, mpqa_dict)
)

df_mcn["tokens"] = df_mcn["tokens_flags"].apply(lambda x: x[0])
df_mcn["flags"] = df_mcn["tokens_flags"].apply(lambda x: x[1])

df_mcn = df_mcn.drop(columns=["tokens_flags"])


100%|██████████| 1000/1000 [00:00<00:00, 6970.95it/s]


In [ ]:
display(df_mcn)

,biased,neutral,tokens,flags
0,"in april 2009 a brazilian human rights group, ...","in april 2009 a brazilian human rights group, ...","[in, april, 2009, a, brazilian, human, rights,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,the 51 day standoff and ensuing murder of 76 m...,the 51 day standoff and ensuing deaths of 76 m...,"[the, 51, day, standoff, and, ensuing, murder,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"mark oaten (born 8 march 1964, watford) is a d...","mark oaten (born 8 march 1964, watford) is a l...","[mark, oaten, (, born, 8, march, 1964, ,, watf...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,another infamous period of colonisation in anc...,another period of colonisation in ancient time...,"[another, infamous, period, of, colonisation, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,photo sequence of astonishing 2005 chicagoland...,photo sequence of 2005 chicagoland crash with ...,"[photo, sequence, of, astonishing, 2005, chica...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
995,especially as the cost to refurbish and modify...,especially as the cost to refurbish and modify...,"[especially, as, the, cost, to, refurbish, and...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
996,blood libels against jews are incorrect accusa...,blood libels against jews are accusations that...,"[blood, libels, against, jews, are, incorrect,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
997,"jacob collier (born august 2, 1994) is a singe...","jacob collier (born august 2, 1994) is a singe...","[jacob, collier, (, born, august, 2, ,, 1994, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
998,they immediately decided to apply it to the an...,they immediately decided to apply it to the ne...,"[they, immediately, decided, to, apply, it, to...","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"


In [ ]:
import os

output_path = os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/biased_word_test.csv")
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df_mcn.to_csv(output_path, index=False)
print("Saved:", output_path)

Saved: /content/drive/MyDrive/CS4650/project/data/WNC/processed/mpqa_lexicon/biased_word_test.csv


In [ ]:
mcn_processed_path=os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/biased_word_test.csv")
df=pd.read_csv(mcn_processed_path)

def format_input(tokens, flags):
    words = tokens.strip("[]").split(", ")
    flags = flags.strip("[]").split(", ")
    words = [w.replace("'", "") for w in words]
    combined = " ".join([f"{w}[{f}]" for w, f in zip(words, flags)])
    return "neutralize: " + combined

df["input_text"] = [format_input(t, f) for t, f in zip(df["tokens"], df["flags"])]
df["target_text"] = df["neutral"]
output_path = os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/input_target_pair_biased_word_test.csv")
output_dir = os.path.dirname(output_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df[["input_text", "target_text"]].to_csv(output_path, index=False)
print("Saved:", output_path)


Saved: /content/drive/MyDrive/CS4650/project/data/WNC/processed/mpqa_lexicon/input_target_pair_biased_word_test.csv


## WNC test

In [ ]:
import torch
from torch import nn
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

In [ ]:
# @title data
dataset_test = load_dataset("csv", data_files=os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/input_target_pair_biased_word_test.csv"))
max_length = 128
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

def preprocess_test(batch):
    inputs = tokenizer(batch["input_text"], max_length=max_length, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["target_text"], max_length=max_length, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

test_dataset = dataset_test.map(preprocess_test, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# @title load model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_path = os.path.join(folder_path,
    "model_checkpoints/wnc-mpqa-flan_t5-2loss-11_09/checkpoint-6000"
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
trained_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
trained_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
# @title bleu score
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
!pip install evaluate
!pip install sacrebleu
import evaluate
import torch

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Ensure predictions and labels are on the correct device and are of the correct type
    preds = torch.tensor(preds, dtype=torch.int64)
    labels = torch.tensor(labels, dtype=torch.int64)

    # Replace potential out-of-vocabulary predicted IDs with pad_token_id
    preds[preds >= tokenizer.vocab_size] = tokenizer.pad_token_id
    preds[preds < 0] = tokenizer.pad_token_id # Also handle potential negative indices

    # Replace -100 with pad_token_id in labels for correct decoding
    labels = labels.clone()
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Ensure references are in the correct format for sacrebleu
    # sacrebleu expects a list of lists for references
    bleu = metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    return {"bleu": bleu["score"]}

args = Seq2SeqTrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,
    report_to="none",
    generation_max_length=128,     # allow up to 128 tokens in output
    generation_num_beams=4,        # optional, improves quality (beam search)
)

trainer = Seq2SeqTrainer(
    model=trained_model,
    args=args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-80161039.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
results = trainer.evaluate(test_dataset)
print(results)


{'eval_train_loss': 0.2901538908481598, 'eval_train_model_preparation_time': 0.0068, 'eval_train_bleu': 69.8724397721992, 'eval_train_runtime': 262.6013, 'eval_train_samples_per_second': 3.808, 'eval_train_steps_per_second': 0.476}


In [ ]:
# @title run predictions on test set

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# make sure predict_with_generate=True so it uses model.generate()
args = Seq2SeqTrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # turn off on GPU
    report_to="none",
    generation_max_length=128,     # allow up to 128 tokens in output
    generation_num_beams=4,        # optional, improves quality (beam search)
)

trainer = Seq2SeqTrainer(
    model=trained_model,
    args=args,
    tokenizer=tokenizer,
)

predictions = trainer.predict(test_dataset["train"])


/tmp/ipython-input-3172449636.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


OverflowError: out of range integral type conversion attempted

In [ ]:

# Replace potential out-of-vocabulary predicted IDs with pad_token_id
predictions.predictions[predictions.predictions >= tokenizer.vocab_size] = tokenizer.pad_token_id
predictions.predictions[predictions.predictions < 0] = tokenizer.pad_token_id # Also handle potential negative indices

preds=tokenizer.batch_decode(
    predictions.predictions, skip_special_tokens=True
)
labels = tokenizer.batch_decode(
    predictions.label_ids, skip_special_tokens=True
)

In [ ]:
import pandas as pd
df_results = pd.DataFrame({
    "input_text": dataset_test["train"]["input_text"],
    "target_text": labels,
    "predicted_text": preds
})
df_results.to_csv(os.path.join(folder_path, "output/WNC/predictions_testset_wnc-mpqa-flan_t5-2loss-11_09_2.csv"), index=False)

display(df_results.head(10))

,input_text,target_text,predicted_text
0,neutralize: in[0] april[0] 2009[0] a[0] brazil...,"in april 2009 a brazilian human rights group, ...","in april 2009 a brazilian human rights group, ..."
1,neutralize: the[0] 51[0] day[0] standoff[0] an...,the 51 day standoff and ensuing deaths of 76 m...,the 51 day standoff and ensuing murder of 76 m...
2,neutralize: mark[0] oaten[0] ([0] born[0] 8[0]...,"mark oaten (born 8 march 1964, watford) is a l...","mark oaten (born 8 march 1964, watford) is a l..."
3,neutralize: another[0] infamous[1] period[0] o...,another period of colonisation in ancient time...,another period of colonisation in ancient time...
4,neutralize: photo[0] sequence[0] of[0] astonis...,photo sequence of 2005 chicagoland crash with ...,photo sequence of 2005 chicagoland crash with ...
5,neutralize: jesus[0] of[0] nazareth[0] is[0] p...,jesus of nazareth is possibly mentioned in two...,jesus of nazareth is probably mentioned in two...
6,neutralize: israeli[0] attempts[0] to[0] reduc...,israeli efforts to reduce gazan civilian casua...,israeli attempts to reduce gazan civilian casu...
7,neutralize: his[0] 45-year[0] career[0] exceed...,his 45-year career was longer than that of any...,his 45-year career exceeded that of any other ...
8,neutralize: they[0] see[0] the[0] calling[0] o...,"they see the calling of such councils, for exa...","they see the calling of such councils, for exa..."
9,neutralize: in[0] the[0] earliest[0] models[0]...,in the earliest models (earlier than a macinto...,in the earliest models (later than a macintosh...


In [ ]:
# @title objectivity evaluation
import numpy as np
clf_name = "GroNLP/mdebertav3-subjectivity-english"
clf_tok = AutoTokenizer.from_pretrained(clf_name)
clf = AutoModelForSequenceClassification.from_pretrained(clf_name).to(trained_model.device).eval()

# predicted text
path=os.path.join(folder_path, "output/WNC/predictions_testset_wnc-mpqa-flan_t5-2loss-11_09_2.csv")
df=pd.read_csv(path)

scores = []
count_objective=0
for text in df["predicted_text"]:
    tokens = clf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
    with torch.no_grad():
        probs = clf(**tokens).logits.softmax(dim=-1)
        p_objective = probs[0][0].item()  # index 0 = objective
        if p_objective>0.5:
          count_objective+=1

    scores.append(p_objective)

print("Mean objectivity score:", np.mean(scores))
print("std:", np.std(scores))
print("total percent of objective sentences:", count_objective/len(scores))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Mean objectivity score: 0.7474374459087849
std: 0.1988511689698736
total percent of objective sentences: 0.858


In [ ]:
# original text:
path=os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/biased_word_test.csv")
df=pd.read_csv(path)
count_objective=0
for text in df["biased"]:
  tokens = clf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
  with torch.no_grad():
      probs = clf(**tokens).logits.softmax(dim=-1)
      p_objective = probs[0][0].item()  # index 0 = objective
      if p_objective>0.5:
          count_objective+=1
  scores.append(p_objective)


print("Mean objectivity score for original text:", np.mean(scores))
print("std:", np.std(scores))
print("total percent of objective sentences:", count_objective/len(scores))

Mean objectivity score for original text: 0.7246217877790332
std: 0.20445795079084508
total percent of objective sentences: 0.4085


In [ ]:
# target text:
path=os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/biased_word_test.csv")
df=pd.read_csv(path)
count_objective=0
for text in df["neutral"]:
  tokens = clf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
  with torch.no_grad():
      probs = clf(**tokens).logits.softmax(dim=-1)
      p_objective = probs[0][0].item()  # index 0 = objective
      if p_objective>0.5:
          count_objective+=1
  scores.append(p_objective)


print("Mean objectivity score for original text:", np.mean(scores))
print("std:", np.std(scores))
print("total percent of objective sentences:", count_objective/len(scores))


Mean objectivity score for original text: 0.7344526325513919
std: 0.20066150870500848
total percent of objective sentences: 0.291


In [ ]:
path=os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/biased_word_test.csv")
df=pd.read_csv(path)
display(df["neutral"])
# for text in df["neutral"]:
#   print(text)

,neutral
0,"in april 2009 a brazilian human rights group, ..."
1,the 51 day standoff and ensuing deaths of 76 m...
2,"mark oaten (born 8 march 1964, watford) is a l..."
3,another period of colonisation in ancient time...
4,photo sequence of 2005 chicagoland crash with ...
...,...
995,especially as the cost to refurbish and modify...
996,blood libels against jews are accusations that...
997,"jacob collier (born august 2, 1994) is a singe..."
998,they immediately decided to apply it to the ne...


In [ ]:
path=os.path.join(folder_path, "output/WNC/predictions_testset_wnc-mpqa-flan_t5-2loss-11_09_2.csv")
df=pd.read_csv(path)
display(df["target_text"])

,target_text
0,"in april 2009 a brazilian human rights group, ..."
1,the 51 day standoff and ensuing deaths of 76 m...
2,"mark oaten (born 8 march 1964, watford) is a l..."
3,another period of colonisation in ancient time...
4,photo sequence of 2005 chicagoland crash with ...
...,...
995,especially as the cost to refurbish and modify...
996,blood libels against jews are accusations that...
997,"jacob collier (born august 2, 1994) is a singe..."
998,they immediately decided to apply it to the ne...


In [ ]:
# target text
path=os.path.join(folder_path, "output/WNC/predictions_testset_wnc-mpqa-flan_t5-2loss-11_09_2.csv")
df=pd.read_csv(path)

scores = []
count_objective=0
for text in df["target_text"]:
    tokens = clf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
    with torch.no_grad():
        probs = clf(**tokens).logits.softmax(dim=-1)
        p_objective = probs[0][0].item()  # index 0 = objective
        if p_objective>0.5:
          count_objective+=1

    scores.append(p_objective)

print("Mean objectivity score:", np.mean(scores))
print("std:", np.std(scores))
print("total percent of objective sentences:", count_objective/len(scores))

Mean objectivity score: 0.7538096265494824
std: 0.19177044989900252
total percent of objective sentences: 0.872


In [ ]:
# Tokenize and decode df["biased"]
path=os.path.join(folder_path, "data/WNC/processed/mpqa_lexicon/biased_word_test.csv")
df=pd.read_csv(path)
biased_tokenized_decoded = []
for text in df["biased"]:
    # Tokenize the text
    tokens = tokenizer(text, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt")
    # Decode the tokens back to text
    decoded_text = tokenizer.batch_decode(tokens["input_ids"], skip_special_tokens=True)
    biased_tokenized_decoded.append(decoded_text[0]) # batch_decode returns a list of lists

# Calculate objectivity score for the tokenized and decoded biased text
scores_biased_processed = []
count_objective_biased_processed = 0
for text in biased_tokenized_decoded:
    tokens = clf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
    with torch.no_grad():
        probs = clf(**tokens).logits.softmax(dim=-1)
        p_objective = probs[0][0].item()  # index 0 = objective
        if p_objective > 0.5:
            count_objective_biased_processed += 1
    scores_biased_processed.append(p_objective)

print("Mean objectivity score for tokenized and decoded biased text:", np.mean(scores_biased_processed))
print("std:", np.std(scores_biased_processed))
print("total percent of objective sentences:", count_objective_biased_processed / len(scores_biased_processed))

Mean objectivity score for tokenized and decoded biased text: 0.70147567717731
std: 0.20773824429799856
total percent of objective sentences: 0.816


# tweets dataset training

## data preprocess

In [ ]:
# @title load and clean

import re

LABEL_HASHTAGS = {
    "sarcasm", "irony", "sarcastic", "ironic",
    "satire", "satirical", "joke", "funny"
}

def clean_tweet(text):
    # lower-case for consistency
    text = text.strip()

    # 1. remove @mentions
    text = re.sub(r"@\w+", "", text)

    # 2. remove URLs
    text = re.sub(r"http\S+|www\.\S+", "", text)

    # 3. process hashtags
    def replace_hashtag(match):
        tag = match.group(1).lower()
        if tag in LABEL_HASHTAGS:
            return ""              # remove style hashtags
        else:
            return tag             # keep content hashtags but drop '#'

    text = re.sub(r"#(\w+)", replace_hashtag, text)

    # 4. collapse repeated spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


In [ ]:
import os
import pandas as pd
from nltk.tokenize import word_tokenize

tweets_path = os.path.join(folder_path, "data/tweets/train.csv")
df_tw = pd.read_csv(tweets_path)

# Keep only non empty tweets
df_tw = df_tw.dropna(subset=["tweets"])
df_tw = df_tw[df_tw["tweets"].str.len() > 3].reset_index(drop=True)


df_tw["tweets"] = df_tw["tweets"].apply(clean_tweet)
df_tw = df_tw[df_tw["tweets"].str.len() > 3]

display(df_tw.head(25))
print(df_tw["class"].value_counts())


,tweets,class
0,Be aware dirty step to get money staylight sta...,figurative
1,for people who don't understand diy artattack,figurative
2,dailymail readers being sensible as always sho...,figurative
3,Why do I get the feeling you like games?,figurative
4,- You probably just missed the text.,figurative
5,Tune in to Nigezie and be treated to Rachel Pl...,figurative
6,What iz thiz?!?!? A friggin DC love fest??!?!?...,figurative
7,"man, i wish i could sexually harass an intoxic...",figurative
8,for the car ride when I get to listen to you a...,figurative
9,Aamir calls bajrangibhaijaan as Salman's best ...,figurative


class
figurative    21229
irony         20884
sarcasm       20674
regular       18595
Name: count, dtype: int64


In [ ]:
# @title flag with lexicon

from tqdm import tqdm
tqdm.pandas()

df_tw["tokens_flags"] = df_tw["tweets"].progress_apply(
    lambda x: tag_subjectivity(x, mpqa_dict)
)
df_tw["tokens"] = df_tw["tokens_flags"].apply(lambda x: x[0])
df_tw["flags"]  = df_tw["tokens_flags"].apply(lambda x: x[1])
df_tw = df_tw.drop(columns=["tokens_flags"])

df_tw[["tweets", "tokens", "flags"]].head(25)


100%|██████████| 81382/81382 [00:06<00:00, 11762.10it/s]


,tweets,tokens,flags
0,Be aware dirty step to get money staylight sta...,"[Be, aware, dirty, step, to, get, money, stayl...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,for people who don't understand diy artattack,"[for, people, who, do, n't, understand, diy, a...","[0, 0, 0, 0, 0, 1, 0, 0]"
2,dailymail readers being sensible as always sho...,"[dailymail, readers, being, sensible, as, alwa...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
3,Why do I get the feeling you like games?,"[Why, do, I, get, the, feeling, you, like, gam...","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]"
4,- You probably just missed the text.,"[-, You, probably, just, missed, the, text, .]","[0, 0, 0, 0, 0, 0, 0, 0]"
5,Tune in to Nigezie and be treated to Rachel Pl...,"[Tune, in, to, Nigezie, and, be, treated, to, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,What iz thiz?!?!? A friggin DC love fest??!?!?...,"[What, iz, thiz, ?, !, ?, !, ?, A, friggin, DC...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
7,"man, i wish i could sexually harass an intoxic...","[man, ,, i, wish, i, could, sexually, harass, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
8,for the car ride when I get to listen to you a...,"[for, the, car, ride, when, I, get, to, listen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,Aamir calls bajrangibhaijaan as Salman's best ...,"[Aamir, calls, bajrangibhaijaan, as, Salman, '...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
def format_input(tokens, flags):
    combined = " ".join(f"{w}[{f}]" for w, f in zip(tokens, flags))
    return "neutralize: " + combined

df_tw["input_text"] = [
    format_input(t, f) for t, f in zip(df_tw["tokens"], df_tw["flags"])
]
df_tw["target_text"] = df_tw["tweets"]

display(df_tw.head(25))


,tweets,class,tokens,flags,input_text,target_text
0,Be aware dirty step to get money staylight sta...,figurative,"[Be, aware, dirty, step, to, get, money, stayl...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",neutralize: Be[0] aware[1] dirty[0] step[0] to...,Be aware dirty step to get money staylight sta...
1,for people who don't understand diy artattack,figurative,"[for, people, who, do, n't, understand, diy, a...","[0, 0, 0, 0, 0, 1, 0, 0]",neutralize: for[0] people[0] who[0] do[0] n't[...,for people who don't understand diy artattack
2,dailymail readers being sensible as always sho...,figurative,"[dailymail, readers, being, sensible, as, alwa...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",neutralize: dailymail[0] readers[0] being[0] s...,dailymail readers being sensible as always sho...
3,Why do I get the feeling you like games?,figurative,"[Why, do, I, get, the, feeling, you, like, gam...","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]",neutralize: Why[0] do[0] I[0] get[0] the[0] fe...,Why do I get the feeling you like games?
4,- You probably just missed the text.,figurative,"[-, You, probably, just, missed, the, text, .]","[0, 0, 0, 0, 0, 0, 0, 0]",neutralize: -[0] You[0] probably[0] just[0] mi...,- You probably just missed the text.
5,Tune in to Nigezie and be treated to Rachel Pl...,figurative,"[Tune, in, to, Nigezie, and, be, treated, to, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: Tune[0] in[0] to[0] Nigezie[0] and...,Tune in to Nigezie and be treated to Rachel Pl...
6,What iz thiz?!?!? A friggin DC love fest??!?!?...,figurative,"[What, iz, thiz, ?, !, ?, !, ?, A, friggin, DC...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...",neutralize: What[0] iz[0] thiz[0] ?[0] ![0] ?[...,What iz thiz?!?!? A friggin DC love fest??!?!?...
7,"man, i wish i could sexually harass an intoxic...",figurative,"[man, ,, i, wish, i, could, sexually, harass, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","neutralize: man[0] ,[0] i[0] wish[1] i[0] coul...","man, i wish i could sexually harass an intoxic..."
8,for the car ride when I get to listen to you a...,figurative,"[for, the, car, ride, when, I, get, to, listen...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: for[0] the[0] car[0] ride[0] when[...,for the car ride when I get to listen to you a...
9,Aamir calls bajrangibhaijaan as Salman's best ...,figurative,"[Aamir, calls, bajrangibhaijaan, as, Salman, '...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: Aamir[0] calls[0] bajrangibhaijaan...,Aamir calls bajrangibhaijaan as Salman's best ...


In [ ]:
# @title save cleaned flagged dataset
all_out = os.path.join(folder_path, "data/tweets/processed/mpqa_lexicon/flagged_train.csv")
os.makedirs(os.path.dirname(all_out), exist_ok=True)
df_tw.to_csv(all_out, index=False)

all_out = os.path.join(folder_path, "data/tweets/processed/mpqa_lexicon/input_target_pair_tweets_train.csv")
os.makedirs(os.path.dirname(all_out), exist_ok=True)
df_tw[["input_text", "target_text"]].to_csv(all_out, index=False)

## Finetune model

In [ ]:
import torch
from torch import nn
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

In [ ]:
# @title load wnc fine tuned model, tokenizer, and clf

# clf:
classifier_name = "GroNLP/mdebertav3-subjectivity-english"
clf_tokenizer = AutoTokenizer.from_pretrained(classifier_name)
clf_model = AutoModelForSequenceClassification.from_pretrained(classifier_name)
clf_model = clf_model.to(device)
clf_model.eval()
for p in clf_model.parameters():
  p.requires_grad = False

# wnc fine tuned model:
checkpoint_path = os.path.join(folder_path,
    "model_checkpoints/wnc-mpqa-flan_t5-2loss-11_09/checkpoint-6000"
)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
model = model.to(device)

In [ ]:
# @title load dataset
from datasets import load_dataset
import os

tweet_train_path = os.path.join(
    folder_path,
    "data/tweets/processed/mpqa_lexicon/input_target_pair_tweets_train.csv"
)

dataset_tw = load_dataset("csv", data_files=tweet_train_path)

print(dataset_tw)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 81382
    })
})


In [ ]:
max_length = 128
def preprocess(batch):
    model_inputs = tokenizer(batch["input_text"], max_length=max_length, truncation=True, padding="max_length")
    labels = tokenizer(batch["target_text"], max_length=max_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_tw = dataset_tw.map(preprocess, batched=True)
print(tokenized_tw)

Map:   0%|          | 0/81382 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 81382
    })
})


In [ ]:
# # @title loss function
# import torch
# from torch import nn
# from transformers import Seq2SeqTrainer
# import torch.nn.functional as F

# class DualLossWithRF(Seq2SeqTrainer):
#     def __init__(self, classifier, clf_tokenizer, lambda_obj=0.2, processing_class=None, **kwargs):
#         super().__init__(**kwargs)
#         self.classifier = classifier
#         self.clf_tokenizer = clf_tokenizer
#         self.lambda_obj = lambda_obj
#         self.processing_class = processing_class
#         self.ce_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

#         # classifier stays frozen
#         self.classifier.eval()
#         for p in self.classifier.parameters():
#             p.requires_grad = False

#     def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
#         # ---------- Forward pass ----------
#         outputs = model(**inputs)
#         logits = outputs.logits
#         labels = inputs["labels"]

#         # ---------- CE loss ----------
#         ce_loss = self.ce_loss_fn(
#             logits.view(-1, logits.size(-1)),
#             labels.view(-1)
#         )

#         # ---------- REINFORCE sampling ----------
#         # sample tokens
#         probs = F.softmax(logits, dim=-1)
#         dist = torch.distributions.Categorical(probs)
#         sampled_ids = dist.sample()   # shape: (batch, seq)

#         # decode sampled text (no gradient needed)
#         sampled_texts = self.processing_class.batch_decode(
#             sampled_ids, skip_special_tokens=True
#         )

#         # ---------- Classifier reward ----------
#         rewards = []
#         for txt in sampled_texts:
#             tokens = self.clf_tokenizer(txt, return_tensors="pt",
#                                         truncation=True, max_length=128).to(model.device)
#             with torch.no_grad():
#                 pred = self.classifier(**tokens).logits.softmax(dim=-1)
#                 reward = pred[0][0]  # objective probability
#                 rewards.append(reward)

#         rewards = torch.stack(rewards).to(model.device)

#         # ---------- REINFORCE loss ----------
#         # log_probability of each sampled token
#         log_probs = torch.log(probs + 1e-12)
#         log_probs_sampled = log_probs.gather(-1, sampled_ids.unsqueeze(-1)).squeeze(-1)

#         rewards = rewards.unsqueeze(1)
#         rl_loss = -(rewards * log_probs_sampled).mean()

#         # ---------- Combined loss ----------
#         total_loss = ce_loss + self.lambda_obj * rl_loss
#         print(f"CE: {ce_loss.item():.4f}, RL: {rl_loss.item():.4f}, TOTAL: {total_loss.item():.4f}")


#         return (total_loss, outputs) if return_outputs else total_loss


In [ ]:
class DualLossWithRF(Seq2SeqTrainer):
    def __init__(self, classifier, clf_tokenizer, lambda_obj=0.2, processing_class=None, **kwargs):
        super().__init__(**kwargs)
        self.classifier = classifier
        self.clf_tokenizer = clf_tokenizer
        self.lambda_obj = lambda_obj
        self.processing_class = processing_class
        self.ce_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

        self.classifier.eval()
        for p in self.classifier.parameters():
            p.requires_grad = False

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

        # ---------- forward ----------
        outputs = model(**inputs)
        logits = outputs.logits
        labels = inputs["labels"]

        # ---------- CE loss ----------
        ce_loss = self.ce_loss_fn(
            logits.view(-1, logits.size(-1)),
            labels.view(-1)
        )

        # ---------- sampling ----------
        probs = torch.softmax(logits, dim=-1)
        dist = torch.distributions.Categorical(probs)
        sampled_ids = dist.sample()

        sampled_texts = self.processing_class.batch_decode(
            sampled_ids, skip_special_tokens=True
        )

        # ---------- classifier reward ----------
        rewards = []
        for txt in sampled_texts:
            tokens = self.clf_tokenizer(
                txt, return_tensors="pt", truncation=True, max_length=128
            ).to(model.device)

            with torch.no_grad():
                pred = self.classifier(**tokens).logits.softmax(dim=-1)
                reward = pred[0][0]
                rewards.append(reward)

        rewards = torch.stack(rewards).to(model.device)  # (batch)

        # ---------- Stable positive RL loss ----------
        log_probs = torch.log(probs + 1e-12)
        log_probs_sampled = log_probs.gather(-1, sampled_ids.unsqueeze(-1)).squeeze(-1)
        seq_log_prob = log_probs_sampled.sum(dim=1)  # (batch)

        rewards = rewards.clamp(0.0, 1.0)

        # always positive penalty
        penalty = (1.0 - rewards) * (-seq_log_prob)
        rl_loss = penalty.mean()

        # ---------- Combined loss ----------
        total_loss = ce_loss + self.lambda_obj * rl_loss

        print(f"CE: {ce_loss.item():.4f}, RL: {rl_loss.item():.4f}, TOTAL: {total_loss.item():.4f}")

        return (total_loss, outputs) if return_outputs else total_loss


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=os.path.join(folder_path, "model_checkpoints/tweets-mpqa-flan_t5-2lossrf-11_16"),
    save_steps=500,
    learning_rate=1e-5,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    weight_decay=0.01,
    predict_with_generate=True,
    report_to="none"
)

trainer = DualLossWithRF(
    model=model,
    processing_class=tokenizer,
    classifier=clf_model,
    clf_tokenizer=clf_tokenizer,
    lambda_obj=0.5,                 # RL weight, between 0.2-0.5 works best
    args=training_args,
    train_dataset=tokenized_tw["train"],
)


In [ ]:
# @title training
trainer.train()

CE: 0.0001, RL: 0.5406, TOTAL: 0.2704


Step,Training Loss
500,0.278700
1000,0.324400
1500,0.282600
2000,0.289700
2500,0.275200
3000,0.274100
3500,0.267000
4000,0.267300
4500,0.201100
5000,0.247200


流式输出内容被截断，只能显示最后 5000 行内容。
CE: 0.0188, RL: 0.0265, TOTAL: 0.0321
CE: 0.0006, RL: 0.4028, TOTAL: 0.2020
CE: 0.0191, RL: 0.0144, TOTAL: 0.0263
CE: 0.0890, RL: 0.1155, TOTAL: 0.1467
CE: 0.0100, RL: 0.0145, TOTAL: 0.0173
CE: 0.0004, RL: 0.1110, TOTAL: 0.0560
CE: 0.0270, RL: 0.1044, TOTAL: 0.0792
CE: 0.0229, RL: 0.0446, TOTAL: 0.0452
CE: 0.0167, RL: 0.1952, TOTAL: 0.1143
CE: 0.0163, RL: 0.0541, TOTAL: 0.0434
CE: 0.0252, RL: 0.0448, TOTAL: 0.0476
CE: 0.0096, RL: 0.0134, TOTAL: 0.0163
CE: 0.0289, RL: 0.1985, TOTAL: 0.1282
CE: 0.0074, RL: 0.2603, TOTAL: 0.1376
CE: 0.0058, RL: 0.0289, TOTAL: 0.0203
CE: 0.0288, RL: 0.1447, TOTAL: 0.1011
CE: 0.0133, RL: 0.0857, TOTAL: 0.0562
CE: 0.0335, RL: 0.1121, TOTAL: 0.0896
CE: 0.0026, RL: 0.9516, TOTAL: 0.4784
CE: 0.0503, RL: 0.0397, TOTAL: 0.0701
CE: 0.0134, RL: 0.0330, TOTAL: 0.0300
CE: 0.0106, RL: 0.6829, TOTAL: 0.3521
CE: 0.0168, RL: 0.0873, TOTAL: 0.0605
CE: 0.0229, RL: 0.1074, TOTAL: 0.0766
CE: 0.0099, RL: 0.0971, TOTAL: 0.0584
CE: 0.0320, RL: 0.1443,

TrainOutput(global_step=10173, training_loss=0.2541352345811504, metrics={'train_runtime': 5076.4435, 'train_samples_per_second': 16.031, 'train_steps_per_second': 2.004, 'total_flos': 1.3931730714230784e+16, 'train_loss': 0.2541352345811504, 'epoch': 1.0})

## Test

In [ ]:
# @title process data
# @title load and clean

import re

LABEL_HASHTAGS = {
    "sarcasm", "irony", "sarcastic", "ironic",
    "satire", "satirical", "joke", "funny"
}

def clean_tweet(text):
    # lower-case for consistency
    text = text.strip()

    # 1. remove @mentions
    text = re.sub(r"@\w+", "", text)

    # 2. remove URLs
    text = re.sub(r"http\S+|www\.\S+", "", text)

    # 3. process hashtags
    def replace_hashtag(match):
        tag = match.group(1).lower()
        if tag in LABEL_HASHTAGS:
            return ""              # remove style hashtags
        else:
            return tag             # keep content hashtags but drop '#'

    text = re.sub(r"#(\w+)", replace_hashtag, text)

    # 4. collapse repeated spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

import os
import pandas as pd
from nltk.tokenize import word_tokenize

tweets_path = os.path.join(folder_path, "data/tweets/test.csv")
df_tw = pd.read_csv(tweets_path)

# Keep only non empty tweets
df_tw = df_tw.dropna(subset=["tweets"])
df_tw = df_tw[df_tw["tweets"].str.len() > 3].reset_index(drop=True)


df_tw["tweets"] = df_tw["tweets"].apply(clean_tweet)
df_tw = df_tw[df_tw["tweets"].str.len() > 3]

display(df_tw.head(25))
print(df_tw["class"].value_counts())



,tweets,class
0,no one ever predicted this was going to happen.,figurative
1,its as closely related as Andrews original cla...,figurative
2,I find it ironic when Vegans say they love food,figurative
3,Quick rt that throwing money vine I've not see...,figurative
4,"yep, keep adding me to your devops lists.... j...",figurative
5,wonderful time with customer service can't add...,figurative
6,: But ... football.,figurative
7,So last month I lost my job and today my dad l...,figurative
8,"Rahm has solved crime, budget &amp; pension is...",figurative
9,And the heel on my shoe just broke. I love whe...,figurative


class
irony         2110
sarcasm       2105
figurative    2043
regular       1859
Name: count, dtype: int64


In [ ]:
# @title flag with lexicon

from tqdm import tqdm
tqdm.pandas()

df_tw["tokens_flags"] = df_tw["tweets"].progress_apply(
    lambda x: tag_subjectivity(x, mpqa_dict)
)
df_tw["tokens"] = df_tw["tokens_flags"].apply(lambda x: x[0])
df_tw["flags"]  = df_tw["tokens_flags"].apply(lambda x: x[1])
df_tw = df_tw.drop(columns=["tokens_flags"])

df_tw[["tweets", "tokens", "flags"]].head(25)

def format_input(tokens, flags):
    combined = " ".join(f"{w}[{f}]" for w, f in zip(tokens, flags))
    return "neutralize: " + combined

df_tw["input_text"] = [
    format_input(t, f) for t, f in zip(df_tw["tokens"], df_tw["flags"])
]
df_tw["target_text"] = df_tw["tweets"]

display(df_tw.head(25))


100%|██████████| 8124/8124 [00:00<00:00, 10264.60it/s]


,tweets,class,tokens,flags,input_text,target_text
0,no one ever predicted this was going to happen.,figurative,"[no, one, ever, predicted, this, was, going, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",neutralize: no[0] one[0] ever[0] predicted[0] ...,no one ever predicted this was going to happen.
1,its as closely related as Andrews original cla...,figurative,"[its, as, closely, related, as, Andrews, origi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",neutralize: its[0] as[0] closely[0] related[0]...,its as closely related as Andrews original cla...
2,I find it ironic when Vegans say they love food,figurative,"[I, find, it, ironic, when, Vegans, say, they,...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0]",neutralize: I[0] find[0] it[0] ironic[1] when[...,I find it ironic when Vegans say they love food
3,Quick rt that throwing money vine I've not see...,figurative,"[Quick, rt, that, throwing, money, vine, I, 'v...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: Quick[0] rt[0] that[0] throwing[0]...,Quick rt that throwing money vine I've not see...
4,"yep, keep adding me to your devops lists.... j...",figurative,"[yep, ,, keep, adding, me, to, your, devops, l...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","neutralize: yep[1] ,[0] keep[0] adding[0] me[0...","yep, keep adding me to your devops lists.... j..."
5,wonderful time with customer service can't add...,figurative,"[wonderful, time, with, customer, service, ca,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",neutralize: wonderful[1] time[0] with[0] custo...,wonderful time with customer service can't add...
6,: But ... football.,figurative,"[:, But, ..., football, .]","[0, 0, 0, 0, 0]",neutralize: :[0] But[0] ...[0] football[0] .[0],: But ... football.
7,So last month I lost my job and today my dad l...,figurative,"[So, last, month, I, lost, my, job, and, today...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",neutralize: So[0] last[0] month[0] I[0] lost[0...,So last month I lost my job and today my dad l...
8,"Rahm has solved crime, budget &amp; pension is...",figurative,"[Rahm, has, solved, crime, ,, budget, &, amp, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",neutralize: Rahm[0] has[0] solved[0] crime[0] ...,"Rahm has solved crime, budget &amp; pension is..."
9,And the heel on my shoe just broke. I love whe...,figurative,"[And, the, heel, on, my, shoe, just, broke, .,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",neutralize: And[0] the[0] heel[0] on[0] my[0] ...,And the heel on my shoe just broke. I love whe...


In [ ]:
# @title save cleaned flagged dataset
all_out = os.path.join(folder_path, "data/tweets/processed/mpqa_lexicon/flagged_test.csv")
os.makedirs(os.path.dirname(all_out), exist_ok=True)
df_tw.to_csv(all_out, index=False)

all_out = os.path.join(folder_path, "data/tweets/processed/mpqa_lexicon/input_target_pair_tweets_test.csv")
os.makedirs(os.path.dirname(all_out), exist_ok=True)
df_tw[["input_text", "target_text"]].to_csv(all_out, index=False)

In [ ]:
# @title load model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_path = os.path.join(folder_path,
    "model_checkpoints/tweets-mpqa-flan_t5-2lossrf-11_16/checkpoint-10000"
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
trained_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
trained_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
# @title load dataset
dataset_test = load_dataset("csv", data_files=os.path.join(folder_path, "data/tweets/processed/mpqa_lexicon/input_target_pair_tweets_test.csv"))
max_length = 128

def preprocess_test(batch):
    inputs = tokenizer(batch["input_text"], max_length=max_length, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["target_text"], max_length=max_length, truncation=True, padding="max_length")
    inputs["labels"] = labels["input_ids"]
    return inputs

test_dataset = dataset_test.map(preprocess_test, batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/8124 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# @title run predictions on test set

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# make sure predict_with_generate=True so it uses model.generate()
args = Seq2SeqTrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # turn off on GPU
    report_to="none",
    generation_max_length=128,     # allow up to 128 tokens in output
    generation_num_beams=4,        # optional, improves quality (beam search)
)

trainer = Seq2SeqTrainer(
    model=trained_model,
    args=args,
    tokenizer=tokenizer,
)

predictions = trainer.predict(test_dataset["train"])

/tmp/ipython-input-946400540.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:

# Replace potential out-of-vocabulary predicted IDs with pad_token_id
predictions.predictions[predictions.predictions >= tokenizer.vocab_size] = tokenizer.pad_token_id
predictions.predictions[predictions.predictions < 0] = tokenizer.pad_token_id # Also handle potential negative indices

preds=tokenizer.batch_decode(
    predictions.predictions, skip_special_tokens=True
)
labels = tokenizer.batch_decode(
    predictions.label_ids, skip_special_tokens=True
)

import pandas as pd
df_results = pd.DataFrame({
    "input_text": dataset_test["train"]["input_text"],
    "target_text": labels,
    "predicted_text": preds
})
df_results.to_csv(os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-2lossrf-11_16.csv"), index=False)

display(df_results.head(10))

,input_text,target_text,predicted_text
0,neutralize: no[0] one[0] ever[0] predicted[0] ...,no one ever predicted this was going to happen.,no one ever predicted this was going to happen.
1,neutralize: its[0] as[0] closely[0] related[0]...,its as closely related as Andrews original cla...,its as closely related as Andrews original cla...
2,neutralize: I[0] find[0] it[0] ironic[1] when[...,I find it ironic when Vegans say they love food,I find it ironic when Vegans say they love food
3,neutralize: Quick[0] rt[0] that[0] throwing[0]...,Quick rt that throwing money vine I've not see...,Quick rt that throwing money vine I've not see...
4,"neutralize: yep[1] ,[0] keep[0] adding[0] me[0...","yep, keep adding me to your devops lists.... j...","yep, keep adding me to your devops lists.... j..."
5,neutralize: wonderful[1] time[0] with[0] custo...,wonderful time with customer service can't add...,wonderful time with customer service can't add...
6,neutralize: :[0] But[0] ...[0] football[0] .[0],: But ... football.,: But... football.
7,neutralize: So[0] last[0] month[0] I[0] lost[0...,So last month I lost my job and today my dad l...,So last month I lost my job and today my dad l...
8,neutralize: Rahm[0] has[0] solved[0] crime[0] ...,"Rahm has solved crime, budget &amp; pension is...","Rahm has solved crime, budget &amp; pension is..."
9,neutralize: And[0] the[0] heel[0] on[0] my[0] ...,And the heel on my shoe just broke. I love whe...,And the heel on my shoe just broke. I love whe...


In [ ]:
# @title objectivity evaluation
import numpy as np
clf_name = "GroNLP/mdebertav3-subjectivity-english"
clf_tok = AutoTokenizer.from_pretrained(clf_name)
clf = AutoModelForSequenceClassification.from_pretrained(clf_name).to(trained_model.device).eval()

# predicted text
path=os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-2lossrf-11_16.csv")
df=pd.read_csv(path)

# mismatches_df = df[df['predicted_text'] != df['target_text']]
# display(mismatches_df)
scores = []
count_objective=0
for text in df["predicted_text"]:
    tokens = clf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
    with torch.no_grad():
        probs = clf(**tokens).logits.softmax(dim=-1)
        p_objective = probs[0][0].item()  # index 0 = objective
        if p_objective>0.5:
          count_objective+=1

    scores.append(p_objective)

print("Mean objectivity score:", np.mean(scores))
print("std:", np.std(scores))
print("total percent of objective sentences:", count_objective/len(scores))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Mean objectivity score: 0.39062714838647067
std: 0.2304217417035224
total percent of objective sentences: 0.26809453471196454


In [ ]:
# target text
path=os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-2lossrf-11_16.csv")
df=pd.read_csv(path)

scores = []
count_objective=0
for text in df["target_text"]:
    tokens = aclf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
    with torch.no_grad():
        probs = clf(**tokens).logits.softmax(dim=-1)
        p_objective = probs[0][0].item()  # index 0 = objective
        if p_objective>0.5:
          count_objective+=1

    scores.append(p_objective)

print("Mean objectivity score:", np.mean(scores))
print("std:", np.std(scores))
print("total percent of objective sentences:", count_objective/len(scores))

Mean objectivity score: 0.38940380347121356
std: 0.23052298472748756
total percent of objective sentences: 0.266494337764648


## load a different checkpoint

In [ ]:
# @title load model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_path = os.path.join(folder_path,
    "model_checkpoints/tweets-mpqa-flan_t5-2lossrf-11_16/checkpoint-4500"
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
trained_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
trained_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
# @title run predictions on test set

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# make sure predict_with_generate=True so it uses model.generate()
args = Seq2SeqTrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # turn off on GPU
    report_to="none",
    generation_max_length=128,     # allow up to 128 tokens in output
    generation_num_beams=4,        # optional, improves quality (beam search)
)

trainer = Seq2SeqTrainer(
    model=trained_model,
    args=args,
    tokenizer=tokenizer,
)

predictions = trainer.predict(test_dataset["train"])

/tmp/ipython-input-946400540.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:

# Replace potential out-of-vocabulary predicted IDs with pad_token_id
predictions.predictions[predictions.predictions >= tokenizer.vocab_size] = tokenizer.pad_token_id
predictions.predictions[predictions.predictions < 0] = tokenizer.pad_token_id # Also handle potential negative indices

preds=tokenizer.batch_decode(
    predictions.predictions, skip_special_tokens=True
)
labels = tokenizer.batch_decode(
    predictions.label_ids, skip_special_tokens=True
)

import pandas as pd
df_results = pd.DataFrame({
    "input_text": dataset_test["train"]["input_text"],
    "target_text": labels,
    "predicted_text": preds
})
df_results.to_csv(os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-2lossrf-11_16-cp4500.csv"), index=False)

display(df_results.head(10))

,input_text,target_text,predicted_text
0,neutralize: no[0] one[0] ever[0] predicted[0] ...,no one ever predicted this was going to happen.,no one ever predicted this was going to happen.
1,neutralize: its[0] as[0] closely[0] related[0]...,its as closely related as Andrews original cla...,its as closely related as Andrews original cla...
2,neutralize: I[0] find[0] it[0] ironic[1] when[...,I find it ironic when Vegans say they love food,I find it ironic when Vegans say they love food
3,neutralize: Quick[0] rt[0] that[0] throwing[0]...,Quick rt that throwing money vine I've not see...,Quick rt that throwing money vine I've not see...
4,"neutralize: yep[1] ,[0] keep[0] adding[0] me[0...","yep, keep adding me to your devops lists.... j...","yep, keep adding me to your devops lists.... j..."
5,neutralize: wonderful[1] time[0] with[0] custo...,wonderful time with customer service can't add...,wonderful time with customer service can't add...
6,neutralize: :[0] But[0] ...[0] football[0] .[0],: But ... football.,: But... football.
7,neutralize: So[0] last[0] month[0] I[0] lost[0...,So last month I lost my job and today my dad l...,So last month I lost my job and today my dad l...
8,neutralize: Rahm[0] has[0] solved[0] crime[0] ...,"Rahm has solved crime, budget &amp; pension is...","Rahm has solved crime, budget &amp; pension is..."
9,neutralize: And[0] the[0] heel[0] on[0] my[0] ...,And the heel on my shoe just broke. I love whe...,And the heel on my shoe just broke. I love whe...


In [ ]:
# @title objectivity evaluation
import numpy as np
clf_name = "GroNLP/mdebertav3-subjectivity-english"
clf_tok = AutoTokenizer.from_pretrained(clf_name)
clf = AutoModelForSequenceClassification.from_pretrained(clf_name).to(trained_model.device).eval()

# predicted text
path=os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-2lossrf-11_16-cp4500.csv")
df=pd.read_csv(path)

# mismatches_df = df[df['predicted_text'] != df['target_text']]
# display(mismatches_df)
scores = []
count_objective=0
for text in df["predicted_text"]:
    tokens = clf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
    with torch.no_grad():
        probs = clf(**tokens).logits.softmax(dim=-1)
        p_objective = probs[0][0].item()  # index 0 = objective
        if p_objective>0.5:
          count_objective+=1

    scores.append(p_objective)

print("Mean objectivity score:", np.mean(scores))
print("std:", np.std(scores))
print("total percent of objective sentences:", count_objective/len(scores))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Mean objectivity score: 0.39084645934450635
std: 0.23076748559242208
total percent of objective sentences: 0.26834071885770555


## load wnc checkpoint

In [ ]:
# @title load model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint_path = os.path.join(folder_path,
    "model_checkpoints/wnc-mpqa-flan_t5-2loss-11_09/checkpoint-6000"
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
trained_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
trained_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
# @title run predictions on test set

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# make sure predict_with_generate=True so it uses model.generate()
args = Seq2SeqTrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # turn off on GPU
    report_to="none",
    generation_max_length=128,     # allow up to 128 tokens in output
    generation_num_beams=4,        # optional, improves quality (beam search)
)

trainer = Seq2SeqTrainer(
    model=trained_model,
    args=args,
    tokenizer=tokenizer,
)

predictions = trainer.predict(test_dataset["train"])

/tmp/ipython-input-946400540.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:

# Replace potential out-of-vocabulary predicted IDs with pad_token_id
predictions.predictions[predictions.predictions >= tokenizer.vocab_size] = tokenizer.pad_token_id
predictions.predictions[predictions.predictions < 0] = tokenizer.pad_token_id # Also handle potential negative indices

preds=tokenizer.batch_decode(
    predictions.predictions, skip_special_tokens=True
)
labels = tokenizer.batch_decode(
    predictions.label_ids, skip_special_tokens=True
)

import pandas as pd
df_results = pd.DataFrame({
    "input_text": dataset_test["train"]["input_text"],
    "target_text": labels,
    "predicted_text": preds
})
df_results.to_csv(os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-2lossrf-11_16-wnc6000.csv"), index=False)

display(df_results.head(10))

,input_text,target_text,predicted_text
0,neutralize: no[0] one[0] ever[0] predicted[0] ...,no one ever predicted this was going to happen.,no one ever predicted this was going to happen.
1,neutralize: its[0] as[0] closely[0] related[0]...,its as closely related as Andrews original cla...,its as closely related as Andrews original cla...
2,neutralize: I[0] find[0] it[0] ironic[1] when[...,I find it ironic when Vegans say they love food,I find it ironic when vegans say they love food
3,neutralize: Quick[0] rt[0] that[0] throwing[0]...,Quick rt that throwing money vine I've not see...,Quick rt that throwing money vine I've not see...
4,"neutralize: yep[1] ,[0] keep[0] adding[0] me[0...","yep, keep adding me to your devops lists.... j...","yep, keep adding me to your devops lists.... j..."
5,neutralize: wonderful[1] time[0] with[0] custo...,wonderful time with customer service can't add...,wonderful time with customer service ca n't ad...
6,neutralize: :[0] But[0] ...[0] football[0] .[0],: But ... football.,: But ... football.
7,neutralize: So[0] last[0] month[0] I[0] lost[0...,So last month I lost my job and today my dad l...,So last month I lost my job and today my dad l...
8,neutralize: Rahm[0] has[0] solved[0] crime[0] ...,"Rahm has solved crime, budget &amp; pension is...","Rahm has solved crime, budget & amp; pension i..."
9,neutralize: And[0] the[0] heel[0] on[0] my[0] ...,And the heel on my shoe just broke. I love whe...,And the heel on my shoe just broke. I love whe...


In [ ]:
display(df_results)

,input_text,target_text,predicted_text
0,neutralize: no[0] one[0] ever[0] predicted[0] ...,no one ever predicted this was going to happen.,no one ever predicted this was going to happen.
1,neutralize: its[0] as[0] closely[0] related[0]...,its as closely related as Andrews original cla...,its as closely related as Andrews original cla...
2,neutralize: I[0] find[0] it[0] ironic[1] when[...,I find it ironic when Vegans say they love food,I find it ironic when vegans say they love food
3,neutralize: Quick[0] rt[0] that[0] throwing[0]...,Quick rt that throwing money vine I've not see...,Quick rt that throwing money vine I've not see...
4,"neutralize: yep[1] ,[0] keep[0] adding[0] me[0...","yep, keep adding me to your devops lists.... j...","yep, keep adding me to your devops lists.... j..."
...,...,...,...
8119,neutralize: Why[0] yes[1] I[0] will[1] totally...,Why yes I will totally submit my photos to a s...,Why yes I will submit my photos to a shitty on...
8120,neutralize: Test[0] on[0] a[0] Saturday[0] ![0...,Test on a Saturday! Thank you uni! @ Griffith ...,Test on a Saturday! Thank you uni! @ Griffith ...
8121,neutralize: Listening[0] to[0] 's[0] Misery[1]...,Listening to 's Misery isn't at all disconcert...,Listening to 's Misery is n't at all disconcer...
8122,neutralize: There[0] you[0] go[0] being[0] kin...,There you go being kind again standup4kids,There you go being kind again standup4kids


In [ ]:
# @title objectivity evaluation
import numpy as np
clf_name = "GroNLP/mdebertav3-subjectivity-english"
clf_tok = AutoTokenizer.from_pretrained(clf_name)
clf = AutoModelForSequenceClassification.from_pretrained(clf_name).to(trained_model.device).eval()

# predicted text
path=os.path.join(folder_path, "output/tweets/predictions_testset_tweets-mpqa-flan_t5-2lossrf-11_16-wnc6000.csv")
df=pd.read_csv(path)

# mismatches_df = df[df['predicted_text'] != df['target_text']]
# display(mismatches_df)
scores = []
count_objective=0
for text in df["predicted_text"]:
    tokens = clf_tok(text, return_tensors="pt", truncation=True).to(trained_model.device)
    with torch.no_grad():
        probs = clf(**tokens).logits.softmax(dim=-1)
        p_objective = probs[0][0].item()  # index 0 = objective
        if p_objective>0.5:
          count_objective+=1

    scores.append(p_objective)

print("Mean objectivity score:", np.mean(scores))
print("std:", np.std(scores))
print("total percent of objective sentences:", count_objective/len(scores))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/881 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Mean objectivity score: 0.40405376168937296
std: 0.24022559221199472
total percent of objective sentences: 0.2917282127031019
